# DiploDatos 2021


### Categorización de publicaciones de productos realizadas en Mercado Libre

### 03 - Introducción al Aprendizaje Automático

#### Condiciones generales que aplican a todos los prácticos:
   - Las notebooks tienen que ser 100% reproducibles, es decir al ejecutar las celdas tal cuál como se entrega la notebook se deben obtener los mismos resultados sin errores.
   - Código legible, haciendo buen uso de las celdas de la notebook y en lo posible seguir estándares de código para *Python* (https://www.python.org/dev/peps/pep-0008/).
   - Utilizar celdas tipo *Markdown* para ir guiando el análisis.
   - Limpiar el output de las celdas antes de entregar el notebook (ir a `Kernel` --> `Restart Kernel and Clear All Ouputs`).
   - Incluir conclusiones del análisis que se hizo en la sección "Conclusiones". Tratar de aportar valor en esta sección, ser creativo.

## 1. Consignas

**Opcional**

Entregar la solución en scripts de *Python*. Se lo puede separar al proyecto en `data.py`, `models.py`, `metrics.py`, `train.py`, etc.

#### Sección 0: División en *train set* y *test set*

- Dividir en entrenamiento (80%) y test (20%).

#### Sección 1:

Tomamos lo generado en el TP anterior. Por lo tanto, partir de lo ya hecho para tener:

- *Sequences*

- *Labels*

- Capa de *Embedding*

(Tener en cuenta que la *tokenización* se hace por separado en los conjuntos de entrenamiento y test, para evitar *leakage* de datos).

#### Sección 2: División en *train set* y *validation set*

- Dividir al conjunto de entrenamiento en entrenamiento y validación utilizando el método de *Stratified K-Folds* (https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.StratifiedKFold.html#sklearn.model_selection.StratifiedKFold)

#### Sección 3: Entrenamiento de un modelo

- Realizar el entrenamiento de un modelo.

- Se aconseja comenzar por una red neuronal *feed forward*, y/o capas del tipo *LSTM*.

- La primera capa de la red será la *embedding_layer* que ya hayan definido y la última capa será una capa densa con `cantidad de neuronas == cantidad de clases` y función de activación *softmax*.

- Entrenar modelos que utilicen tanto los embeddings pre-entrenados como los customs.

- Experimentar utilizando *dropout* y *batch normalization*.

- Utilizar *checkpoints* para guardar el modelo.

- Utilizar como *loss* la *sparse_categorical_crossentropy*, y se recomienda utilizar *Adam* como *optimizer*.

#### Sección 4: Evaluación del modelo

- Graficar curvas de *loss* y *accuracy* para entrenamiento y validación (se puede hacer utilizando el *model history* de *Keras*, https://www.tensorflow.org/api_docs/python/tf/keras/callbacks/History).

- Utilizar *balance accuracy* para medir la performance del modelo en validación y test, https://scikit-learn.org/stable/modules/generated/sklearn.metrics.balanced_accuracy_score.html.

- Diferenciar el *score* según *label_quality*. Es decir, calcular el puntaje para el conjunto de validación y test separando según *reliable* y *unreliable*.

#### Sección 5: Análisis de resultados

- Realizar análisis de que modelo performó mejor, generar un `.csv` con los resultados en validación y test de cada modelo.

#### Sección 6: Exportado de predicciones

- Generar un *dataframe* con las predicciones del conjunto de test del mejor modelo, y exportarlo a un `.csv`.

## 2. Código y análisis

Instalaciones necesarias

In [1]:
!pip install Unidecode

Importaciones necesarias

In [2]:
import pandas as pd
import numpy as np

Lectura de dataset reducido

In [3]:
df_dataset = pd.read_csv('DataSet/dataset.csv')

Estudiamos el dataset brevemente antes de comenzar a operar sobre el mismo.

In [4]:
df_dataset.describe()

,title,label_quality,language,category
count,646760,646760,646760,646760
unique,646019,2,2,20
top,Teclado Yamaha Ypt-230,unreliable,portuguese,PANTS
freq,2,551878,328992,35973


In [5]:
classes = np.sort(df_dataset.category.unique())

print(f'Dimensiones: {df_dataset.shape}')
print('----------')
print(f'Variables: {list(df_dataset.columns)}')
print('----------')
print(f'Categorías: {list(classes)}')

Dimensiones: (646760, 4)
----------
Variables: ['title', 'label_quality', 'language', 'category']
----------
Categorías: ['BABY_CAR_SEATS', 'BABY_STROLLERS', 'COFFEE_MAKERS', 'ELECTRIC_DRILLS', 'HAIR_CLIPPERS', 'KITCHEN_SINKS', 'MATTRESSES', 'MEMORY_CARDS', 'MOTORCYCLE_JACKETS', 'MUSICAL_KEYBOARDS', 'PANTS', 'PUREBRED_DOGS', 'RANGES', 'REFRIGERATORS', 'ROLLER_SKATES', 'SEWING_MACHINES', 'SHORTS', 'SUITCASES', 'WALL_CLOCKS', 'WINES']


**Sección 0**

In [6]:
import re
from unidecode import unidecode

def cleaning(title):
    """
    Aplica las operaciones de limpieza a un título de una publicación.
    """
    # Unidecode: Convierte string de Unicode a ASCII.
    title = unidecode(title)
    # Pasamos a Minúsculas.
    title = title.lower()
    # Eliminamos Números.
    title = re.sub(r'[0-9]+', '', title)
    # Reemplazamos Contracciones.
    title = re.sub(r'c/u', 'cada uno', title)
    title = re.sub(r'c/', 'con', title)
    title = re.sub(r'p/', 'para', title)
    # Limpiamos Símbolos.
    title = re.sub('[^a-zA-Z ]', '', title)
    # Retornamos el título de la publicación procesado.
    return title

In [7]:
df_dataset['clean_title'] = df_dataset.title.apply(cleaning)

In [8]:
from sklearn.model_selection import train_test_split

df_train, df_test = train_test_split(df_dataset, train_size=0.8, test_size=0.2, random_state=123)

print(f'Dimensiones Entrenamiento: {df_train.shape}')
print(f'Dimensiones Evaluación: {df_test.shape}')

Dimensiones Entrenamiento: (517408, 5)
Dimensiones Evaluación: (129352, 5)


In [9]:
# Datos de Entrenamiento
X_train = df_train.clean_title.values
y_train = df_train.category.values
# Datos de Test
X_test = df_test.clean_title.values
y_test = df_test.category.values

**Sección 1**

In [10]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

word_tokenizer = Tokenizer()
# Aprendemos el Tokenizer en base a los datos de entrenamiento.
word_tokenizer.fit_on_texts(X_train)
# Longitud del Vocabulario en base a los datos de entrenamiento.
vocab_length = len(word_tokenizer.word_index) + 1

embedded_sentences_train = word_tokenizer.texts_to_sequences(X_train)
embedded_sentences_test = word_tokenizer.texts_to_sequences(X_test)

padded_sentences_train = pad_sequences(embedded_sentences_train, padding='post')
# Longitud de Sentencias en base a los datos de entrenamiento.
ammount_sentences, sentences_length = padded_sentences_train.shape

padded_sentences_test = pad_sequences(embedded_sentences_test, padding='post', maxlen=sentences_length)

In [11]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
le.fit(classes)

encoded_labels_train = le.transform(y_train)
encoded_labels_test = le.transform(y_test)

In [12]:
from keras.layers.embeddings import Embedding

dense_vector_size = 30
embedding_layer = Embedding(vocab_length, dense_vector_size, input_length=sentences_length)

**Sección 2**

In [ ]:
# TO DO

## 3. Conclusiones

In [ ]:
# TO DO